In [ ]:
# Install Gradio for UI
!pip install gradio


In [ ]:
# Basic imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical


upload GTSRB datase

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("meowmeowmeowmeowmeow/gtsrb-german-traffic-sign")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/gtsrb-german-traffic-sign


In [ ]:
!mv "kaggle (4).json" kaggle.json

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json  # Permissions සකසන්න

In [ ]:
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign
!unzip -q gtsrb-german-traffic-sign.zip  # -q = quiet mode (no output)

Dataset URL: https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign
License(s): CC0-1.0
User cancelled operation
replace Meta.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Preprocess the Data

In [ ]:
data_dir = '/content/Train'
images = []
labels = []

IMG_HEIGHT = 32
IMG_WIDTH = 32

for label in os.listdir(data_dir):
    path = os.path.join(data_dir, label)
    if not os.path.isdir(path):
        continue
    for img_file in os.listdir(path):
        img_path = os.path.join(path, img_file)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
        images.append(img)
        labels.append(label)

images = np.array(images)
labels = np.array(labels).astype('int')


Normalize and one-hot encode

In [ ]:
images = images / 255.0
labels = to_categorical(labels)


Split into train/test

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)


Define CNN Model

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(43, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Train the Model

In [ ]:
history = model.fit(X_train, y_train, epochs=70, validation_data=(X_val, y_val))


Epoch 1/70
981/981 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.3385 - loss: 2.4708 - val_accuracy: 0.9022 - val_loss: 0.4175
Epoch 2/70
981/981 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - accuracy: 0.7993 - loss: 0.6374 - val_accuracy: 0.9612 - val_loss: 0.1832
Epoch 3/70
981/981 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8821 - loss: 0.3781 - val_accuracy: 0.9751 - val_loss: 0.1100
Epoch 4/70
981/981 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9118 - loss: 0.2794 - val_accuracy: 0.9805 - val_loss: 0.0868
Epoch 5/70
981/981 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9317 - loss: 0.2135 - val_accuracy: 0.9860 - val_loss: 0.0611
Epoch 6/70
981/981 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9425 - loss: 0.1789 - val_accuracy: 0.9872 - val_loss: 0.0545
Epoch 7/70
981/981 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9512 - loss: 0.1543 - val_accuracy: 0.9888 - val_loss: 0.0474
Epoch 8/70
981/981 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9559 - loss: 0.1395 - val_accuracy: 

Evaluate model

In [ ]:
!pip install gradio

import gradio as gr

def classify_traffic_sign(img):
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    img = np.expand_dims(img, axis=0) / 255.0
    pred = model.predict(img)
    class_id = np.argmax(pred)
    return f"Predicted Class: {class_id}"

gr.Interface(fn=classify_traffic_sign, inputs="image", outputs="text").launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9a15defdad91f61e95.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr

# Example mapping (replace with actual class names for GTSRB dataset)
class_id_to_name = {
    0: "Speed Limit 20",
    1: "Speed Limit 30",
    2: "Speed Limit 50",
    3: "Speed Limit 60",
    4: "Speed Limit 70",
    5: "Speed Limit 80",
    6: "End Speed Limit 80",
    7: "Speed Limit 100",
    8: "Speed Limit 120",
    9: "No Passing",
    10: "No Passing for Vehicles Over 3.5 Tons",
    11: "Right-of-Way at Next Intersection",
    12: "Priority Road",
    13: "Yield",
    14: "Stop",
    # Add all 43 classes here (replace with actual names from GTSRB)
}

def classify_traffic_sign(img):
    # Preprocess the image
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    img = np.expand_dims(img, axis=0) / 255.0

    # Predict
    pred = model.predict(img)
    class_id = np.argmax(pred)

    # Get sign name and representation
    sign_name = class_id_to_name.get(class_id, f"Class {class_id}")
    representation = "STOP" if class_id == 14 else "YIELD" if class_id == 13 else str(class_id)  # Example logic

    # Format output
    return f"{sign_name}:\n{representation}"

# Launch Gradio interface
gr.Interface(
    fn=classify_traffic_sign,
    inputs="image",
    outputs="text",
    title="Traffic Sign Recognition",
    description="Upload a traffic sign image to classify it."
).launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://270cb541429a65c46a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
